In [36]:
import pandas as pd
import seaborn as sns
from bs4 import BeautifulSoup
from bs4.element import Comment
import urllib.request
from urllib.request import URLError
import requests
%matplotlib inline
sns.set()

In [2]:
df = pd.read_csv('./free_company_dataset.csv', error_bad_lines=False)

b'Skipping line 69743: expected 10 fields, saw 11\nSkipping line 101632: expected 10 fields, saw 11\n'
b'Skipping line 157754: expected 10 fields, saw 11\nSkipping line 189017: expected 10 fields, saw 11\n'
b'Skipping line 308592: expected 10 fields, saw 11\nSkipping line 312800: expected 10 fields, saw 11\n'
b'Skipping line 333445: expected 10 fields, saw 11\nSkipping line 353899: expected 10 fields, saw 11\nSkipping line 361250: expected 10 fields, saw 11\nSkipping line 374180: expected 10 fields, saw 11\nSkipping line 374658: expected 10 fields, saw 11\n'
b'Skipping line 416636: expected 10 fields, saw 11\nSkipping line 435987: expected 10 fields, saw 11\nSkipping line 455847: expected 10 fields, saw 11\n'
b'Skipping line 463320: expected 10 fields, saw 11\nSkipping line 481278: expected 10 fields, saw 11\nSkipping line 500441: expected 10 fields, saw 11\nSkipping line 504289: expected 10 fields, saw 11\n'
b'Skipping line 530687: expected 10 fields, saw 11\nSkipping line 542886: exp

In [3]:
df.head()

,country,founded,id,industry,linkedin_url,locality,name,region,size,website
0,united kingdom,2017.0,codingcodex,education management,linkedin.com/company/codingcodex,nottingham,codex,nottingham,11-50,codingwithcodex.co.uk
1,italy,2016.0,rid968fm,broadcast media,linkedin.com/company/rid968fm,rome,rid 96.8 fm,lazio,1-10,rid968.com
2,costa rica,NaN,eclipse-in-action,consumer services,linkedin.com/company/eclipse-in-action,san josé,eclipse in action,san jose,201-500,supportseven.com
3,turkey,2008.0,pergel-digital-media,internet,linkedin.com/company/pergel-digital-media,NaN,pergel digital media,NaN,1-10,pergeldigital.com
4,united states,2007.0,cougar-electric-llc,construction,linkedin.com/company/cougar-electric-llc,swartz creek,cougar electric llc,michigan,1-10,power-now.biz


In [4]:
df.country.nunique(), df.locality.nunique(), df.region.nunique()

(249, 90979, 4443)

In [5]:
df.isna().sum()/df.shape[0]*100

country         16.297923
founded         50.180549
id               0.000010
industry         9.042004
linkedin_url     0.000000
locality        33.002053
name             0.748760
region          28.927910
size             0.000000
website         24.894256
dtype: float64

In [6]:
#print(df.country.nunique())
country_df = df.groupby('country').count().sort_values('id', ascending=False)[['id', 'website']]
country_df.columns = ['n_companies', 'n_websites']
print(country_df.shape[0])
country_df.head(10)

249


,n_companies,n_websites
country,,
united states,3213829,2396446
united kingdom,935786,635134
india,396714,288193
brazil,308647,249234
france,278142,173166
canada,267602,228169
australia,262869,233152
netherlands,247567,214751
germany,234000,210501


In [7]:
industry_df = df.groupby('industry').count().sort_values('id', ascending=False)[['id', 'website']]
industry_df.columns = ['n_companies', 'n_websites']
print(industry_df.shape[0])
industry_df.head(20)

147


,n_companies,n_websites
industry,,
information technology and services,534903,451873
construction,427072,305335
marketing and advertising,385629,335856
real estate,274137,204571
management consulting,242218,194122
computer software,225451,179958
financial services,214856,167027
retail,201062,146664
"health, wellness and fitness",196334,163582


In [11]:
df['english'] = df.country.map(lambda x: x in ['united states', 'united kingdom', 'australia', 'canada'])
clean_df = df[(~df.website.isna()) & df.english & (~df.industry.isna())& (~df.locality.isna())& (~df.region.isna())]
clean_df.to_csv('free_company_dataset_clean.csv')
clean_df.shape

(2802311, 11)

In [15]:
%%time
def tag_visible(element):
    if element.parent.name in ['style', 'script', 'head', 'title', 'meta', '[document]']:
        return False
    if isinstance(element, Comment):
        return False
    return True


def text_from_html(body):
    soup = BeautifulSoup(body, 'html.parser')
    texts = soup.findAll(text=True)
    visible_texts = filter(tag_visible, texts)  
    return u" ".join(t.strip() for t in visible_texts)


def parse_websites(values):
    dirty_texts = {}
    clean_texts = {}
    for website, id_ in values:
        try:
            response = requests.get('https://{}/'.format(website), allow_redirects=False)
            mystr = response.content
            dirty_texts[id_] = mystr
            clean_texts[id_] = text_from_html(mystr)
        except requests.ConnectionError as e:
            continue
        except requests.TooManyRedirects as e:
            continue
    return dirty_texts, clean_texts

r = parse_websites(clean_df[['website', 'id']].values[:100])

Wall time: 4min 12s


In [28]:
import json
import nltk
from bs4 import BeautifulSoup
import re
import seaborn as sns

In [22]:
clean_text.strip()

'Home About Our Team Careers Services Civil Design Case Study: Distribution Center Case Study: Big Box Retail Site Design Case Study: Market Grocery Site Design Land Planning Case Study: Camp Jackson Case Study: Crossroads Regional Park Case Study: Whispering Woods Surveying Drone Services Forensic Engineering Case Study: Expansive Clay Mitigation Case Study: Retaining Wall Failure Case Study: Storm Sewer Collapse Environmental Case Study: Creek Relocation Case Study: Membership Wholesale Club Case Study: Wetlands Mitigation Landscape Architecture Case Study: Brighton Ball Field Case Study: Park of Dreams Case Study: Webb F. Banks Passive Park Clients Contact \xa0(901) 384-0404 CIVIL DESIGNLAND PLANNINGSURVEYINGDRONE SERVICESFORENSIC ENGINEERINGENVIRONMENTALLANDSCAPE ARCHITECTURE Carlson Consulting Engineers, Inc. Welcome to Carlson Consulting Engineers. We are leaders in civil engineering, planning, design and development for both large and small clients. Founded in 1995, our firm has

In [37]:
with open('./Lab/200_companies.json', 'r') as f:
    a = json.load(f)
len(a)

3063

In [31]:
clean_texts = []
for d in a:
    soup = BeautifulSoup(d['text'], 'html.parser')
    clean_text = re.sub(' +', ' ',soup.get_text().replace('\n', ' ')).strip().split(' ')
    clean_texts.append(clean_text)

In [38]:
lengths = [len(x['text']) for x in a]
sum([x > 50 for x in lengths]), sum([x > 100 for x in lengths]), sum([x > 200 for x in lengths]), 

(3063, 2921, 2553)

In [40]:
targets = pd.read_csv('./Lab/to_parse.csv')[['website']]
targets.head()

,website
0,codingwithcodex.co.uk
1,power-now.biz
2,miracledoors.com
3,carlsonconsulting.net
4,beaconmarketinggroupinc.com


In [45]:
from math import ceil
CHUNK_SIZE = 200000
n_chunks = ceil(len(targets)/CHUNK_SIZE)
for i in range(n_chunks):
    targets.iloc[i*CHUNK_SIZE:(i+1)*CHUNK_SIZE].to_csv('./Lab/targets/{:2f}.csv'.format(i), index=False)

In [47]:
'./Lab/targets/{:2d}.csv'.format(1)

'./Lab/targets/ 1.csv'

In [44]:
targets.iloc[i*CHUNK_SIZE:(i+1)*CHUNK_SIZE]

,website
0,codingwithcodex.co.uk
1,power-now.biz
2,miracledoors.com
3,carlsonconsulting.net
4,beaconmarketinggroupinc.com
...,...
199995,acao.org
199996,3dpt.com
199997,whalesrus.com
199998,nursingunlimitedonline.com
